In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.5 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=7c365b50ed03b5cbced1e5d648cfd3d4c9bb76651f9d75da1a26fec2f9551bd2
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.6 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

ln: failed to create symbolic link '/mydrive/MyDrive': File exists
'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [5]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [6]:
!ls

 a0529-a0611_US		   heatmap529_UK.ipynb
 a0529.csv		   heatmap530.ipynb
 a0530.csv		   Heat_Map_Online_language.ipynb
 csv			   __MACOSX
 csv.zip		   non_violant1.csv
 dataframe_csv		   non_violant2.csv
 dataframe_csv_UK	   UK
 dataframe_result_csv	   violant.csv
 dataframe_result_csv_UK  '新建 Microsoft PowerPoint 演示文稿.pptx'
 Github


In [7]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
ct_path_531 = 'UK/05_31.csv'

In [9]:
ct = pd.read_csv(ct_path_531)

In [10]:
ct.shape[1]

47

In [11]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [12]:
#len(columns1)
len(columns2)

47

In [13]:
ct.columns=columns2

In [14]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,52552983299_10157916498178300,Facebook,2020-05-31 23:59:56,2021-08-31 12:54:55,link,George Floyd Death Protesters Return To Capito...,denver.cbslocal.com,"Smart: ""We stand for the truth. We stand for j...",Protesters returned to the Colorado State Capi...,1.0,https://denver.cbslocal.com/2020/05/31/george-...,https://www.facebook.com/52552983299/posts/101...,502230,1.463158,1.0,34,7,41,15,2,2,7,31,0,0,19,8,22,12,5,7,9,13,0,4,20339,CBS Denver,CBSDenver,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Covering Colorado First. Your #1 source for br...,True,2009-02-25 23:34:30,en,0,20339|10157916498178300
1,57427307078_10159355042537079,Facebook,2020-05-31 23:59:53,2021-08-31 20:39:39,link,Psychology professor explains how race factore...,abc7news.com,"Smart: ""We stand for the truth. We stand for j...",What started out as a mostly peaceful demonstr...,1.0,https://abc7news.com/6222900/?ex_cid=TA_KGO_FB...,https://www.facebook.com/57427307078/posts/101...,1219608,-5.451613,1.0,14,5,6,1,0,2,0,3,0,0,50,15,41,9,9,10,21,14,0,6,19637,ABC7 News,abc7news,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Get breaking news and the most talked about st...,True,2009-01-26 18:11:53,en,0,19637|10159355042537079
2,129617873765147_3588816481178585,Facebook,2020-05-31 23:59:53,2021-06-20 22:23:37,link,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Follow our latest updates on the riots as tens...,1.0,https://www.express.co.uk/news/world/1289602/g...,https://www.facebook.com/129617873765147/posts...,1768821,-3.893333,1.0,17,8,34,0,1,2,2,11,0,0,96,14,105,15,5,29,8,20,0,6,89685,Daily Express,DailyExpress,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,The Official Facebook page of the Daily and Su...,True,2010-12-10 10:07:29,en,0,89685|3588816481178585
3,821532787927852_3008252662589176,Facebook,2020-05-31 23:59:52,2021-07-09 03:56:24,native_video,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Are You a Uniter or a Divider - Bible lesson T...,21.0,https://www.facebook.com/GunGuyTV/videos/25336...,https://www.facebook.com/821532787927852/posts...,4813,-2.000000,2.0,11,3,0,0,0,0,0,0,0,0,10,5,2,4,1,2,1,3,0,1,11480437,Gunguytv,GunGuyTV,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Find out more about the GunGuy at https://joel...,False,2015-05-01 05:50:26,en,0,11480437|3008252662589176
4,123264117724844_3292223030828921,Facebook,2020-05-31 23:59:49,2021-06-27 10:36:41,photo,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Trumbull officials issue statement on George F...,2.0,https://www.facebook.com/TrumbullTimes/photos/...,https://www.facebook.com/123264117724844/posts...,3621,1.695652,1.0,30,2,0,7,0,0,0,0,0,0,6,3,5,6,1,0,2,0,0,0,28309,The Trumbull Times,TrumbullTimes,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Your leader in Trumbull news, sports, business...",False,2010-09-02 13:22:50,en,0,28309|3292223030828921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [15]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [16]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [17]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [20]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [21]:
ct['NER-msg'] = ct['message'].apply(ner)

In [22]:
ct['NER-desc'] = ct['description'].apply(ner)

In [23]:
ct['NER-title'] = ct['title'].apply(ner)

In [24]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Denver': 1},{}
1,{'Los Angeles': 1},{}
2,{},"{'Minneapolis': 1, 'US': 1}"
3,{},"{'Minneapolis': 1, 'US': 1}"
4,{},"{'Minneapolis': 1, 'US': 1}"


In [25]:
ct.to_csv('a0531.csv')

In [26]:
ct=pd.read_csv('a0531.csv')

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Denver': 1},{},{'Denver': 1}
1,{'Los Angeles': 1},{},{'Los Angeles': 1}
2,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
3,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
4,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

0                        [Denver]
1                   [Los Angeles]
2                              []
3                              []
4                              []
                   ...           
39014               [Minneapolis]
39015                          []
39017    [L.A., Los Angeles City]
39018                          []
39019                          []
Name: NER-msg, Length: 30749, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

0                                         [Denver, Denver]
1                               [Los Angeles, Los Angeles]
2                           [US, Minneapolis, Minneapolis]
3                           [US, Minneapolis, Minneapolis]
4                           [US, Minneapolis, Minneapolis]
                               ...                        
39014    [Minneapolis, Memphis, Los Angeles, Minneapoli...
39015                                        [Los Angeles]
39017    [L.A., Los Angeles City, L.A., Fairfax, Minnea...
39018                              [Portland, Minneapolis]
39019                                        [Los Angeles]
Name: NER, Length: 30749, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['denver']), list(['los angeles']),
       list(['us', 'minneapolis']), ...,
       list(['l.a.', 'los angeles city', 'minneapolis', 'fairfax', 'los angeles sentinel']),
       list(['portland', 'minneapolis']), list(['los angeles'])],
      dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['denver',
 'los angeles',
 'us',
 'minneapolis',
 'us',
 'minneapolis',
 'us',
 'minneapolis',
 'minneapolis',
 'illinois',
 'oak park',
 'oak park 123',
 'chicago',
 'hayward',
 'minneapolis',
 'cape girardeau',
 'cape girardeau',
 'brooklyn',
 'nyc',
 'minneapolis',
 'minneapolis',
 'sacramento',
 'sacramento',
 'chicago',
 'hollywood',
 'maine',
 'portland',
 'vermont',
 'eugene',
 'maine',
 'vermont',
 'portland',
 'oregon',
 'us',
 'mississippi',
 'us',
 'neshoba county',
 'minneapolis',
 'us',
 'minneapolis',
 'salt lake city',
 'minneapolis',
 'america',
 'salt lake city',
 'minneapolis',
 'salt lake city',
 'san jose',
 'philando castile',
 'minneapolis',
 'san jose',
 'seattle',
 'the united states',
 'san jose',
 'las vegas',
 'u.s.',
 'san jose',
 'israel',
 'palestine',
 'san jose',
 'mississippi',
 'san jose',
 'neshoba county',
 'minneapolis',
 'los angeles county',
 'minneapolis',
 'new jersey',
 'america',
 'minneapolis',
 'miami',
 'america',
 'michigan',
 'flint twp'

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('minneapolis', 12746),
 ('us', 3896),
 ('minnesota', 3726),
 ('america', 3248),
 ('u.s.', 2048),
 ('los angeles', 1948),
 ('the united states', 1773),
 ('new york', 1177),
 ('chicago', 1130),
 ('london', 1076),
 ('atlanta', 954),
 ('washington', 638),
 ('houston', 593),
 ('seattle', 576),
 ('denver', 564),
 ('philadelphia', 528),
 ('miami', 521),
 ('california', 510),
 ('usa', 467),
 ('texas', 455),
 ('new york city', 434),
 ('genesee county', 428),
 ('santa cruz', 418),
 ('dallas', 402),
 ('brooklyn', 398),
 ('michigan', 375),
 ('flint', 367),
 ('georgia', 359),
 ('oakland', 355),
 ('tulsa', 328),
 ('louisville', 313),
 ('pittsburgh', 311),
 ('uk', 276),
 ('berlin', 273),
 ('detroit', 272),
 ('colorado', 266),
 ('nyc', 259),
 ('florida', 258),
 ('nashville', 254),
 ('ohio', 246),
 ('mn', 245),
 ('d.c.', 238),
 ('flint township', 238),
 ('portland', 226),
 ('richmond', 218),
 ('germany', 213),
 ('flint twp', 211),
 ('cleveland', 206),
 ('columbus', 205),
 ('indianapolis', 203),
 ('ok

In [43]:
dataframe531=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

3788

In [45]:
dataframe531.to_csv('dataframe0531_UK.csv')